In [1]:
# importing relavent packages
import numpy as np
import pandas as pd
from os import listdir
import os
import glob
## function for data loading
def DataETL(datapath):
    """
    DataETL loads experimental data from all experimental data files (xlsx).
    Filters data and keeps only important columns.
    Combine selected data and save it to csv file.

    useage: DataETL('datapath')
    """
    data_filenames_list = glob.glob((datapath + 'php_*.xlsx'))
    df_frames = []
    for i in range(0, len(data_filenames_list)) :
        df_raw = pd.read_excel((data_filenames_list[i]))
        selected_columns = ['Time (Min)', 'Tc - AVG (oC)', 'Te - AVG (oC)', 'Pressure (mm of Hg)', 'Te - Tc (oC)', 'Q (W)',
            'Resistance (oC/W)']
        df_selected_columns = df_raw[selected_columns]
        df_frames.append(df_selected_columns)
        df = pd.concat(df_frames, axis=0, ignore_index=True)
        df_out = df.to_csv(datapath + "combined_data.csv")
    return df


In [2]:
# data sorting and selecting

def DataChop(data, Tmin=0, Tmax=100):
    """ 
    DataChop function used to chop the data for the selected temperature value from the Te_avg column.

    useage: DataChop(df, Tmin, Tmax)
    here, Tmin/Tmax is a suitable value (int) from the data.
    """
    assert 0 < Tmin < Tmax, f"Enter correct value of Tmin"
    assert Tmin < Tmax < 100, f"Enter correct value of Tmax"

    data_T = data[data['Te - AVG 1,2,3 (oC)'] <= Tmax]
    data_T = data_T[data_T['Te - AVG 1,2,3 (oC)'] >= Tmin]
    return data_T

In [1]:
# data mixing and re-arranging

def DataArrange(data, path):
    """
    DataArrange sorts and arrange value by group, calculates mean and standard deviation of the grouped data.
    Calculated result will be stored at the location of data file.

    DataArrange(data, path)
    """
    df_mean = data.sort_values(by=['Te - AVG (oC)']).groupby(['Te - AVG (oC)'], as_index=False).mean()
    df_mean_out = df_mean.to_csv(path + 'combined_mean.csv')
    df_std = data.sort_values(by=['Te - AVG (oC)']).groupby(['Te - AVG (oC)'], as_index=False).std().dropna()
    df_std_out = df_std.to_csv(path + 'combined_std.csv')
    return df_mean, df_std

In [ ]:
# claculation of avg values of Tc, Te, P, dT, TR

def DataPropAvg(df_mean, df_std):
    # avg values 
    Tc_avg = df_mean['Tc - AVG (oC)'].mean()
    P_avg = df_mean['Pressure (mm of Hg)'].mean()
    dT_avg = df_mean['Te - Tc (oC)'].mean()
    TR_avg = df_mean['Resistance (oC/W)'].mean()
    # std values
    Tc_std = df_std['Tc - AVG (oC)'].mean()
    P_std = df_std['Pressure (mm of Hg)'].mean()
    dT_std = df_std['Te - Tc (oC)'].mean()
    TR_std = df_std['Resistance (oC/W)'].mean()
    
    msg = (f"Tc  average:     {round(Tc_avg,4)} +- {round(Tc_std,4)} [C]\n"
    f"P   average:     {round(P_avg,4)} +- {round(P_std,4)} [mmHg]\n"
    f"dT  average:     {round(dT_avg,4)} +- {round(dT_std,4)} [C]\n"
    f"TR  average:     {round(TR_avg,4)} +- {round(TR_std,4)} [C/W]\n");
    #msg_out = msg.format(Tc_avg, Tc_std, Te_avg, Te_std, P_avg, P_std, dT_avg, dT_std, TR_avg, TR_std)
    return print(msg)